In [ ]:
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/
!git checkout lmdb

Cloning into 'Generative_Models'...
remote: Enumerating objects: 2077, done.
remote: Counting objects: 100% (2077/2077), done.
remote: Compressing objects: 100% (592/592), done.
remote: Total 14901 (delta 1504), reused 1906 (delta 1367), pack-reused 12824
Receiving objects: 100% (14901/14901), 414.67 MiB | 35.49 MiB/s, done.
Resolving deltas: 100% (10105/10105), done.
Checking out files: 100% (1675/1675), done.
/content/Generative_Models
Already on 'lmdb'
Your branch is up to date with 'origin/lmdb'.


In [ ]:
!pip -q install keras
!pip -q install colorlog
!pip -q install keras-radam

In [ ]:
import os
os.chdir('/content/Generative_Models/')
print(os.getcwd())

/content/Generative_Models


# Dataset loading

In [ ]:
dataset_name='pokemon'

In [ ]:
images_dir = '/content/Generative_Models/data/.pokemon/'
validation_percentage = 30
valid_format = 'png'

In [ ]:
from training.generators.file_image_generator import create_image_lists, get_generators

In [ ]:
!ls -la /content/Generative_Models/data/.pokemon

total 28
drwxr-xr-x 3 root root  4096 Jun 19 17:03 .
drwxr-xr-x 6 root root  4096 Jun 19 17:03 ..
drwxr-xr-x 2 root root 20480 Jun 19 17:03 DS06


In [ ]:
image_lists = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format,
    verbose = 1
)

  DEBUG    | Looking for images in 'DS06'
  INFO     | 809 file found


In [ ]:
import numpy as np
import lmdb
import pickle
import math
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img

In [ ]:
from utils.data_and_files.file_utils import get_file_path

In [ ]:
class Pokemon_Image:
    def __init__(self, image, label):
        # Dimensions of image for reconstruction - not really necessary 
        # for this dataset, but some datasets may include images of 
        # varying sizes
        self.channels = image.shape[2]
        self.size = image.shape[:2]

        self.image = image.tobytes()
        self.label = label

    def get_images(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.uint8)
        return images.reshape(-1,*self.size, self.channels)

    def get_image(self):
        """ Returns the image as a numpy array. """
        images = np.frombuffer(self.image, dtype=np.uint8)
        # print(images.shape)
        return images.reshape(np.array(self.size, self.channels))

In [ ]:
def store_many_lmdb(filename, images, labels):
    """ Stores an array of images to LMDB.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    #str(lmdb_dir / f"{num_images}_lmdb")
    env = lmdb.open(filename, map_size=map_size, create= True)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            value = Pokemon_Image(images[i], labels[i])
            key = f"{i:08}"
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

In [ ]:
import sys
def store_single_lmdb(filename, wrapper , index, num_images):
    """ Stores a wrapper to LMDB.
    """

    map_size = wrapper.get_image().nbytes * num_images + sys.getsizeof(wrapper.label) * 2 * num_images
    env = lmdb.open(filename, map_size=map_size, create= True)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
          # All key-value pairs need to be Strings
          value = wrapper
          key = f"{index:08}"
          txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()

Test 1

In [ ]:
!rm -rf /content/LMDB/
!mkdir /content/LMDB/

In [ ]:
lmdb_dir = '/content/LMDB'

category='training'

# print('num_class',num_class)
image_dir=images_dir

target_size=None
color_mode='rgb'
# save_to_dir=None,
save_prefix=''
save_format='png'

def store_lmdb(image_lists, image_dir, 
               lmdb_dir = '/content/LMDB',category='training', target_size=None,
               color_mode='rgb', save_prefix='', save_format='png'):
  classes = list(image_lists.keys())
  num_class = len(classes)
  class2id = dict(zip(classes, range(len(classes))))
  id2class = dict((v, k) for k, v in class2id.items())
 
  y = None
  X = None

  for label_name in classes:
    num_images = len(image_lists[label_name][category])
    for index, _ in enumerate(image_lists[label_name][category]):
        img_path = get_file_path(image_lists,
                                  label_name,
                                  index,
                                  image_dir,
                                  category)
        img = img_to_array(
                        load_img(
                        img_path,
                        grayscale=color_mode=='grayscale',
                        target_size=target_size
                        )
                    )
        name, _ = os.path.splitext(img_path)
        vid_img_arr = name.split(sep=os.sep)[-1:]
        y = [np.array(vid_img_arr)]
        X = np.array(img)
        # print(X.shape)
        # print(X.shape[2])
        # print(X.shape[:2])
        
        name =  lmdb_dir+os.sep+'_{}'.format(category)
        pokemon_Image = Pokemon_Image(X, y)

        store_single_lmdb(index = index, filename=name, wrapper = pokemon_Image, num_images = num_images)
        

        # if (y is None):
        #       y = [np.array(vid_img_arr)]
        # else:
        #       y = np.append(y, [np.array(vid_img_arr)], 0)
        # if (X is None):
        #       X = [np.array(img)]
        # else:
        #       X = np.append(X, [img], 0)
        # if (index+1) % 200 == 0 and index != 0:
        #     print('Loaded {} 200 pictures'.format((index+1) / 200))
        #     print('Saving images as LMDB file')
        #     store_many_lmdb(lmdb_dir+os.sep+'_{}'.format((index+1) / 200), images = X, labels = y)
        #     y = None
        #     X = None
        # elif len(image_lists[label_name][category])-1 == index:
        #     print('Loaded last {} pictures'.format(math.ceil((index+1) / 200)))
        #     print('Saving images as LMDB file')
        #     store_many_lmdb(lmdb_dir+os.sep+'_{}'.format(float(math.ceil((index+1) / 200))), images = X, labels = y)
        #     y = None
        #     X = None
            


In [ ]:
store_lmdb(image_lists, image_dir)

TypeError: ignored

In [ ]:
# In  case we want each label to be a key then

def store_many_lmdb_labelaskey(filename, images, labels):
    """ Stores an array of images to LMDB.
        Parameters:
        ---------------
        images       images array, (N, 32, 32, 3) to be stored
        labels       labels array, (N, 1) to be stored
    """
    num_images = len(images)

    map_size = num_images * images[0].nbytes * 10

    # Create a new LMDB DB for all the images
    #str(lmdb_dir / f"{num_images}_lmdb")
    env = lmdb.open(filename, map_size=map_size)

    # Same as before — but let's write all the images in a single transaction
    with env.begin(write=True) as txn:
        for i in range(num_images):
            # All key-value pairs need to be Strings
            value = Pokemon_Image(images[i], labels[i])
            key = labels[i]
            txn.put(key.encode("ascii"), pickle.dumps(value))
    env.close()


In [ ]:
import math
def read_many_lmdb(filename, num_images):
    """ Reads image from LMDB.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []
    #str(lmdb_dir / f"{num_images}_lmdb")
    for folder_indx in  range(math.ceil(num_images/200)):
      env = lmdb.open(filename+os.sep+'_{}'.format(float(folder_indx+1)), readonly=True)
      print(folder_indx)
      # Start a new read transaction
      with env.begin() as txn:
          # Read all images in one single transaction, with one lock
          # We could split this up into multiple transactions if needed
          if num_images- folder_indx*200 >200:
            max_index = 200 
          else:
            max_index = num_images - folder_indx*200
          
          for image_id in range(0,max_index):
              data = txn.get(f"{image_id:08}".encode("ascii"))
              # Remember that it's a CIFAR_Image object 
              # that is stored as the value
              pokemon_image = pickle.loads(data)
              # Retrieve the relevant bits
              images.append(pokemon_image.get_images())
              labels.append(pokemon_image.label)
      env.close()
    return images, labels

Test 2 Reading data


In [ ]:
X, y = read_many_lmdb(lmdb_dir, 809)

In [ ]:
print()

In [ ]:
def store_lmdb(
             lmdb_name,
             image_lists,
             category, 
             image_dir,
             target_size=None,
             color_mode='rgb',
             save_to_dir=None,
             save_prefix='',
             save_format='png'
):

    classes = list(image_lists.keys())
    num_class = len(classes)

    class2id = dict(zip(classes, range(len(classes))))
    id2class = dict((v, k) for k, v in class2id.items())

    if color_mode not in {'rgb', 'grayscale'}:
        raise ValueError('Invalid color mode:', color_mode, '; expected "rgb" or "grayscale".')

    lmdb_env = lmdb.open(lmdb_name)
    # lmdb_db = lmdb_env.begin(write=True)
    # print(classes)
    with lmdb_env.begin(write=True) as lmdb_db:
      for label_name in classes:
          for j, _ in enumerate(image_lists[label_name][category]):
              img_path = get_file_path(image_lists,
                                        label_name,
                                        j,
                                        image_dir,
                                        category)
              img = img_to_array(
                              load_img(
                              img_path,
                              grayscale=color_mode=='grayscale',
                              target_size=target_size
                              )

                          )
              
              str_id.encode('ascii'), datum.SerializeToString()
              X.tobytes()

              
              lmdb_db.put(label_name.encode(), img, append=True, overwrite=False)      
    lmdb_env.close()

In [ ]:
store_lmdb(lmdb_name='pokemon2', image_lists=imgs_list, category='training', image_dir=images_dir)

NameError: ignored

In [ ]:
lmdb_env = lmdb.open('pokemon2')
with lmdb_env.begin() as lmdb_txn:
    with lmdb_txn.cursor() as lmdb_cursor:
        for key, value in lmdb_cursor:  
           print (key)
           print (value)

In [ ]:
keys = [key for key, _ in lmdb_db.cursor() ]
values = [value for _, value in lmdb_db.cursor() ]

In [ ]:
print(np.fromstring(values))

In [ ]:
import numpy as np
import lmdb

def write_lmdb(filename):
    print ('Write lmdb')

    lmdb_env = lmdb.open(filename, map_size=int(1e9))

    n_samples= 2
    X= (255*np.random.rand(n_samples,3,4,3)).astype(np.uint8)
    y= np.random.rand(n_samples).astype(np.float32)

    for i in range(n_samples):
        with lmdb_env.begin(write=True) as lmdb_txn:
            lmdb_txn.put('X_'+str(i), X)
            lmdb_txn.put('y_'+str(i), y)

            print ('X:',X)
            print ('y:',y)

def read_lmdb(filename):
    print ('Read lmdb')

    lmdb_env = lmdb.open(filename)
    lmdb_txn = lmdb_env.begin()
    lmdb_cursor = lmdb_txn.cursor()

    n_samples=0
    with lmdb_env.begin() as lmdb_txn:
        with lmdb_txn.cursor() as lmdb_cursor:
            for key, value in lmdb_cursor:  
                print (key)
                if('X' in key):
                    print (np.fromstring(value, dtype=np.uint8))
                if('y' in key):
                    print (np.fromstring(value, dtype=np.float32))

                n_samples=n_samples+1

    print ('n_samples',n_samples)

write_lmdb('temp.db')
read_lmdb('temp.db')

Implemented in code

In [ ]:
import glob
import hashlib
import logging
import os
import re
import warnings

from keras.preprocessing.image import ImageDataGenerator

from utils.data_and_files.data_utils import as_bytes
from utils.reporting.logging import log_message
from .image_iterator import ImageIterator


class FileImageGenerator(ImageDataGenerator):
    def flow_from_image_lists(self, image_lists,
                              category,
                              image_dir,
                              target_size,
                              batch_size,
                              episode_len=None,
                              episode_shift=None,
                              color_mode='rgb',
                              class_mode=None,
                              shuffle=True,
                              seed=None,
                              save_to_dir=None,
                              save_prefix='',
                              save_format='jpg'):

        return ImageIterator(image_lists, self,
                             category,
                             image_dir,
                             target_size=target_size,
                             color_mode=color_mode,
                             class_mode=class_mode,
                             data_format=self.data_format,
                             batch_size=batch_size,
                             episode_len=episode_len,
                             episode_shift=episode_shift,
                             shuffle=shuffle, seed=seed,
                             save_to_dir=save_to_dir,
                             save_prefix=save_prefix,
                             save_format=save_format)


def create_image_lists(image_dir, validation_pct, valid_imgae_formats, max_num_images_per_class=2**27-1, verbose = 1):
    """Builds a list of training images from the file system.

    Analyzes the sub folders in the image directory, splits them into stable
    training, testing, and validation sets, and returns a data structure
    describing the lists of images for each label and their paths.

    # Arguments
        image_dir: string path to a folder containing subfolders of images.
        validation_pct: integer percentage of images reserved for validation.

    # Returns
        dictionary of label subfolder, with images split into training
        and validation sets within each label.
    """
    if not os.path.isdir(image_dir):
        raise ValueError("Image directory {} not found.".format(image_dir))
    image_lists = {}
    sub_dirs = [x[0] for x in os.walk(image_dir)]

    sub_dirs_without_root = sub_dirs[1:]  # first element is root directory
    for sub_dir in sub_dirs_without_root:
        file_list = []
        dir_name = os.path.basename(sub_dir)
        if dir_name == image_dir:
            continue
        if verbose == 1:
            log_message("Looking for images in '{}'".format(dir_name), logging.DEBUG)

        if isinstance(valid_imgae_formats, str):
            valid_imgae_formats = [valid_imgae_formats]

        for extension in valid_imgae_formats:
            file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list:
            msg = 'No files found'
            if verbose == 1:
                log_message(msg, logging.WARN)
            warnings.warn(msg)
            continue
        else:
            if verbose == 1:
                log_message('{} file found'.format(len(file_list)), logging.INFO)
        if len(file_list) < 20:
            msg = 'Folder has less than 20 images, which may cause issues.'
            if verbose == 1:
                log_message(msg, logging.WARN)
            warnings.warn(msg)
        elif len(file_list) > max_num_images_per_class:
            msg='WARNING: Folder {} has more than {} images. Some '\
                          'images will never be selected.' \
                          .format(dir_name, max_num_images_per_class)
            log_message(msg, logging.WARN)
            warnings.warn(msg)
        label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
        training_images = []
        validation_images = []
        for file_name in file_list:
            base_name = os.path.basename(file_name)
            # Get the hash of the file name and perform variant assignment.
            hash_name = hashlib.sha1(as_bytes(base_name)).hexdigest()
            hash_pct = ((int(hash_name, 16) % (max_num_images_per_class  + 1)) *
                        (100.0 / max_num_images_per_class))
            if hash_pct < validation_pct:
                validation_images.append(base_name)
            else:
                training_images.append(base_name)
        image_lists[label_name] = {
            'dir': dir_name,
            'training': training_images,
            'validation': validation_images,
        }
    return image_lists

def get_generators(images_list, image_dir, image_size, batch_size, class_mode, episode_len=None, episode_shift=None, scaler=255.0):

    train_datagen = FileImageGenerator(rescale=1. / scaler)

    valid_datagen = FileImageGenerator(rescale=1. / scaler)

    train_generator = train_datagen.flow_from_image_lists(
        image_lists=images_list,
        category='training',
        image_dir=image_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=class_mode,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    validation_generator = valid_datagen.flow_from_image_lists(
        image_lists=images_list,
        category='validation',
        image_dir=image_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode=class_mode,
        episode_len=episode_len,
        episode_shift=episode_shift,
        seed=0)

    return train_generator, validation_generator

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import Iterator, load_img, img_to_array, array_to_img
from keras import backend as K
import logging
from utils.reporting.logging import log_message
from utils.data_and_files.file_utils import get_file_path

class ImageIterator(Iterator):
    """Iterator capable of reading images from a directory on disk.

    # Arguments
        image_lists: Dictionary of training images for each label.
        image_data_generator: Instance of `ImageDataGenerator`
            to use for random transformations and normalization.
        target_size: tuple of integers, dimensions to resize input images to.
        color_mode: One of `"rgb"`, `"grayscale"`. Color mode to read images.
        classes: Optional list of strings, names of sudirectories
            containing images from each class (e.g. `["dogs", "cats"]`).
            It will be computed automatically if not set.
        class_mode: Mode for yielding the targets:
            `"binary"`: binary targets (if there are only two classes),
            `"categorical"`: categorical targets,
            `"sparse"`: integer targets,
            `None`: no targets get yielded (only input images are yielded).
            `episode`: a sequence of images yielded (with time shift).
            `func`: custom transformation gets the input images and the transformed.

        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        data_format: String, one of `channels_first`, `channels_last`.
        save_to_dir: Optional directory where to save the pictures
            being yielded, in a viewable format. This is useful
            for visualizing the random transformations being
            applied, for debugging purposes.
        save_prefix: String prefix to use for saving sample
            images (if `save_to_dir` is set).
        save_format: Format to use for saving sample images
            (if `save_to_dir` is set).
    """

    def __init__(self,
                 image_lists,
                 image_data_generator,
                 category, image_dir,
                 target_size=(256, 256, 3),
                 color_mode='rgb',
                 class_mode='categorical',
                 batch_size=32,
                 episode_len=20,
                 episode_shift=10,
                 shuffle=True,
                 seed=None,
                 data_format=None,
                 save_to_dir=None,
                 save_prefix='',
                 save_format='jpeg',
                 dtype=K.floatx()
                 ):
        if data_format is None:
            data_format = K.image_data_format()

        classes = list(image_lists.keys())
        self.category = category
        self.batch_size = batch_size
        self.num_class = len(classes)
        self.image_lists = image_lists
        self.image_dir = image_dir
        self.episode_len = episode_len
        self.episode_shift = episode_shift

        how_many_files = 0
        for label_name in classes:
            for _ in self.image_lists[label_name][category]:
                how_many_files += 1

        self.samples = how_many_files
        self.class2id = dict(zip(classes, range(len(classes))))
        self.id2class = dict((v, k) for k, v in self.class2id.items())
        self.classes = np.zeros((self.samples,), dtype='int32')

        self.image_data_generator = image_data_generator
        self.target_size = tuple(target_size)
        if color_mode not in {'rgb', 'grayscale'}:
            raise ValueError('Invalid color mode:', color_mode,
                             '; expected "rgb" or "grayscale".')
        self.color_mode = color_mode
        self.data_format = data_format
        self.image_shape = self.target_size

        if (class_mode not in {'categorical', 'binary', 'sparse', 'episode', 'episode_flat', None}) and (not hasattr(class_mode, '__call__')):
            raise ValueError('Invalid class_mode:', class_mode,
                             '; expected one of "categorical", '
                             '"binary", "sparse", "episode", or None.')
        self.class_mode = class_mode
        self.dtype = dtype
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format

        i = 0
        self.filenames = []
        for label_name in classes:
            for j, _ in enumerate(self.image_lists[label_name][category]):
                self.classes[i] = self.class2id[label_name]
                img_path = get_file_path(self.image_lists,
                                          label_name,
                                          j,
                                          self.image_dir,
                                          self.category)
                self.filenames.append(img_path)
                i += 1
        log_message("Found {} {} files".format(len(self.filenames), category), logging.INFO)
        Iterator.__init__(self, self.samples, self.batch_size, shuffle, seed)

    def _get_batches_of_transformed_samples(self, index_array):
        """For python 2.x.

        # Returns
            The next batch.
        """
        # with self.lock:
        #    index_array, current_index, current_batch_size = next(self.index_generator)

        # The transformation of images is not under thread lock
        # so it can be done in parallel

        if len(index_array) < self.batch_size:
            diff = self.batch_size//len(index_array) + 1
            index_array = np.repeat(index_array, diff, axis=0)[:self.batch_size]

        grayscale = self.color_mode == 'grayscale'
        if self.class_mode == 'episode':
            batch_x = np.zeros((len(index_array), self.episode_len) + self.image_shape, dtype=self.dtype)
            batch_gt = np.zeros((len(index_array), self.episode_len) + self.image_shape, dtype=self.dtype)

            def get_filename(path):
                folder, file = path.split(os.path.sep)[-2:]
                file_name = file.split('.')[0]
                return folder, int(file_name)


            sorted_filenames = sorted(self.filenames, key=lambda f: get_filename(f))

            max_ix = int(get_filename(sorted_filenames[-1])[1])
            last_ix = max_ix - (self.episode_len + self.episode_shift)

            # build batch of image data
            for i, j in enumerate(index_array):
                if j > last_ix:
                    j = j - last_ix

                imgs = []
                for ix in range(j, j+self.episode_len):
                    imgs += [
                        self.image_data_generator.standardize(
                            img_to_array(
                                load_img(
                                    sorted_filenames[ix],
                                    grayscale=grayscale,
                                    target_size=self.target_size
                                ),
                                data_format=self.data_format
                            )
                        )
                    ]
                imgs = np.array(imgs)
                batch_x[i] = imgs

                imgs = []
                for ix in range(j+self.episode_shift, j+self.episode_len+self.episode_shift):
                    imgs += [
                        self.image_data_generator.standardize(
                            img_to_array(
                                load_img(
                                    sorted_filenames[ix],
                                    grayscale=grayscale,
                                    target_size=self.target_size
                                ),
                                data_format=self.data_format
                            )
                        )
                    ]

                imgs = np.array(imgs)
                batch_gt[i] = imgs
            return batch_x, batch_gt

        elif self.class_mode == 'episode_flat':
            batch_x = np.zeros((len(index_array), self.episode_len) + self.image_shape, dtype=self.dtype)
            batch_gt = np.zeros((len(index_array), self.episode_len) + self.image_shape, dtype=self.dtype)

            def get_filename(path):
                folder, file = path.split(os.path.sep)[-2:]
                file_name = file.split('.')[0]
                return folder, int(file_name)

            sorted_filenames = sorted(self.filenames, key=lambda f: get_filename(f))

            max_ix = int(get_filename(sorted_filenames[-1])[1])
            last_ix = max_ix - (self.episode_len + self.episode_shift)

            # build batch of image data
            for i, j in enumerate(index_array):
                if j > last_ix:
                    j = j - last_ix

                imgs = []
                for ix in range(j, j + self.episode_len):
                    imgs += [
                        self.image_data_generator.standardize(
                            img_to_array(
                                load_img(
                                    sorted_filenames[ix],
                                    grayscale=grayscale,
                                    target_size=self.target_size
                                ),
                                data_format=self.data_format
                            )
                        )
                    ]
                imgs = np.array(imgs)
                batch_x[i] = imgs

                imgs = []
                for ix in range(j + self.episode_shift, j + self.episode_len + self.episode_shift):
                    imgs += [
                        self.image_data_generator.standardize(
                            img_to_array(
                                load_img(
                                    sorted_filenames[ix],
                                    grayscale=grayscale,
                                    target_size=self.target_size
                                ),
                                data_format=self.data_format
                            )
                        )
                    ]

                imgs = np.array(imgs)
                batch_gt[i] = imgs
            return np.reshape(batch_x, (-1,)+self.image_shape ), np.reshape(batch_gt, (-1,)+self.image_shape)
        else:
            batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=self.dtype)
            # build batch of image data
            for i, j in enumerate(index_array):
                img = load_img(self.filenames[j],
                               grayscale=grayscale,
                               target_size=self.target_size)
                x = img_to_array(img, data_format=self.data_format)
                x = self.image_data_generator.random_transform(x)
                x = self.image_data_generator.standardize(x)
                batch_x[i] = x

            # optionally save augmented images to disk for debugging purposes
            if self.save_to_dir:
                for i, j in enumerate(index_array):
                    img = array_to_img(batch_x[i], self.data_format, scale=True)
                    fname = '{prefix}_{index}_{hash}.{format}'.format(prefix=self.save_prefix,
                                                                      index=j,
                                                                      hash=np.random.randint(10000),
                                                                      format=self.save_format)
                    img.save(os.path.join(self.save_to_dir, fname))
            # build batch of labels
            if self.class_mode == 'sparse':
                batch_y = self.classes[index_array]
            elif self.class_mode == 'binary':
                batch_y = self.classes[index_array].astype(K.floatx())
            elif self.class_mode == 'categorical':
                batch_y = np.zeros((len(batch_x), self.num_class),dtype=K.floatx())
                for i, label in enumerate(self.classes[index_array]):
                    batch_y[i, label] = 1.

            elif self.class_mode is None:
                return batch_x
            else:
                return batch_x, self.class_mode(batch_x)
            return batch_x, batch_y

